Based on: https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-create-a-neural-network-for-regression-with-pytorch.md

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [ ]:
torch.manual_seed(42)

In [ ]:
X, y = fetch_california_housing(return_X_y=True)

In [ ]:
class HousingDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            # Apply scaling if necessary
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X.astype(np.float32))
            self.y = torch.from_numpy(y.astype(np.float32))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [ ]:
dataset = HousingDataset(X, y)
trainloader = torch.utils.data.DataLoader(
    dataset, batch_size=10, shuffle=True, num_workers=1)

In [ ]:
next(iter(trainloader))

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Initialize the MLP
mlp = MLP()

# Define the loss function and optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [ ]:
# Run the training loop
for epoch in range(0, 5):  # 5 epochs at maximum

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):

        # Get and prepare inputs
        inputs, targets = data
        targets = targets.reshape((targets.shape[0], 1))

        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = mlp(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 200 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 500))
            current_loss = 0.0

# Process is complete.
print('Training process has finished.')

### Save Model

In [ ]:
torch.save(mlp, "housing_model.pt")

### Load and Test Model

In [ ]:
loaded_mlp = torch.load("housing_model.pt")

In [ ]:
testX, testY = next(iter(trainloader))

In [ ]:
loaded_mlp(testX).flatten()

In [ ]:
testY

### Columns as separate input variables

In [ ]:
import numpy as np
import pandas as pd
import torch

from inspect import signature
from torch import nn
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [ ]:
torch.manual_seed(42)

In [ ]:
housing = fetch_california_housing()

In [ ]:
class HousingDataset2(torch.utils.data.Dataset):
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            # Apply scaling if necessary
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X.astype(np.float32))
            self.y = torch.from_numpy(y.astype(np.float32))
            
            # Split dataset into separate variables
            self.MedInc = self.X[:,0]
            self.HouseAge = self.X[:,1]
            self.AveRooms = self.X[:,2]
            self.AveBedrms = self.X[:,3]
            self.Population = self.X[:,4]
            self.AveOccup = self.X[:,5]
            self.Latitude = self.X[:,6]
            self.Longitude = self.X[:,7]

    def __len__(self):
        return len(self.MedInc)

    def __getitem__(self, i):
        # Note: also returning combined X for ease of use later
        return self.MedInc[i], self.HouseAge[i], self.AveRooms[i], self.AveBedrms[i], self.Population[i], self.AveOccup[i], self.Latitude[i], self.Longitude[i], self.y[i]

In [ ]:
dataset2 = HousingDataset2(housing.data, housing.target)
trainloader2 = torch.utils.data.DataLoader(dataset2, batch_size=10, shuffle=True, num_workers=1)

In [ ]:
next(iter(trainloader2))

In [ ]:
class MLP2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, inc, age, rms, bdrms, pop, occup, lat, lon):       
        combined = torch.column_stack((inc, age, rms, bdrms, pop, occup, lat, lon))
        return self.layers(combined)

In [ ]:
# Initialize the MLP
mlp2 = MLP2()

In [ ]:
# Define the loss function and optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(mlp2.parameters(), lr=1e-4)

In [ ]:
# Run the training loop
for epoch in range(0, 5):  # 5 epochs at maximum

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader2, 0):

        # Get and prepare inputs
        a,b,c,d,e,f,g,h,targets = data
        targets = targets.reshape((targets.shape[0], 1))

        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = mlp2(a,b,c,d,e,f,g,h)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 200 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 500))
            current_loss = 0.0

# Process is complete.
print('Training process has finished.')

In [ ]:
torch.save(mlp2, "housing_model2.pt")

In [ ]:
a,b,c,d,e,f,g,h,targets = next(iter(trainloader2))

In [ ]:
mlp2(a,b,c,d,e,f,g,h)

In [ ]:
print(signature(mlp2.forward))

## PySpark

### Convert dataset to Spark DataFrame

In [ ]:
housing = fetch_california_housing()

In [ ]:
X = StandardScaler().fit_transform(housing.data.astype(np.float32))

In [ ]:
pdf = pd.DataFrame(X, columns=housing.feature_names)
pdf

In [ ]:
pdf.dtypes

In [ ]:
from pyspark.sql.types import *

# Spark is somehow auto-converting Pandas float32 to DoubleType(), so forcing FloatType()
schema = StructType([
StructField("MedInc",FloatType(),True),
StructField("HouseAge",FloatType(),True),
StructField("AveRooms",FloatType(),True),
StructField("AveBedrms",FloatType(),True),
StructField("Population",FloatType(),True),
StructField("AveOccup",FloatType(),True),
StructField("Latitude",FloatType(),True),
StructField("Longitude",FloatType(),True)
])

df = spark.createDataFrame(pdf, schema=schema)

In [ ]:
df.schema

In [ ]:
df.show(truncate=12)

### Save DataFrame as parquet

In [ ]:
df.write.mode("overwrite").parquet("california_housing")

## Inference using Spark ML Model
Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

In [ ]:
import sparkext
import torch

In [ ]:
df = spark.read.parquet("california_housing")

In [ ]:
df.show(truncate=12)

In [ ]:
columns = df.columns
columns

In [ ]:
from torch import nn

class MLP2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, inc, age, rms, bdrms, pop, occup, lat, lon):       
        combined = torch.column_stack((inc, age, rms, bdrms, pop, occup, lat, lon))
        return self.layers(combined)

In [ ]:
my_model = sparkext.torch.Model("housing_model2.pt") \
                .setInputCols(columns) \
                .setOutputCol("preds")

In [ ]:
predictions = my_model.transform(df)

In [ ]:
predictions.show()

## Inference using Spark DL UDF
Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

In [ ]:
import torch

from pyspark.sql.functions import col
from sparkext.torch import model_udf

In [ ]:
df = spark.read.parquet("california_housing")

In [ ]:
df.show(truncate=12)

In [ ]:
columns = df.columns
columns

### Using Saved Model

Since the model is pickled, the model class must be defined before loading.

In [ ]:
from torch import nn

class MLP2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, inc, age, rms, bdrms, pop, occup, lat, lon):       
        combined = torch.column_stack((inc, age, rms, bdrms, pop, occup, lat, lon))
        return self.layers(combined)

In [ ]:
classify = model_udf("housing_model2.pt", input_columns=columns)

In [ ]:
predictions = df.withColumn("preds", classify(*columns))

In [ ]:
%%time
preds = predictions.collect()

In [ ]:
predictions.show(truncate=12)